# Configuración del ambiente

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import requests
from bs4 import BeautifulSoup

# Obtención de datos

## Precios Históricos del Bitcoin

In [33]:
def importar_base_bitcoin():
    global df_bitcoin, precio_actual, tendencia, media_bitcoin,  algoritmo_decision
    datos_bitcoin = yf.Ticker('BTC-USD')
    df_bitcoin = pd.DataFrame(datos_bitcoin.history(period = '7d', interval = '5m'))

importar_base_bitcoin()
df_bitcoin.head(10)
    

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-10-21 00:00:00+00:00,29683.380859,29688.470703,29677.050781,29677.050781,0,0.0,0.0
2023-10-21 00:05:00+00:00,29685.591797,29697.873047,29685.591797,29697.873047,0,0.0,0.0
2023-10-21 00:10:00+00:00,29694.681641,29694.681641,29678.056641,29678.056641,0,0.0,0.0
2023-10-21 00:15:00+00:00,29679.595703,29701.451172,29679.595703,29687.312500,0,0.0,0.0
2023-10-21 00:20:00+00:00,29681.755859,29683.751953,29673.628906,29682.761719,606208,0.0,0.0
2023-10-21 00:25:00+00:00,29683.175781,29697.013672,29683.175781,29697.013672,7835648,0.0,0.0
2023-10-21 00:30:00+00:00,29701.640625,29701.640625,29678.408203,29678.408203,0,0.0,0.0
2023-10-21 00:35:00+00:00,29677.173828,29678.009766,29667.054688,29667.054688,0,0.0,0.0
2023-10-21 00:40:00+00:00,29666.908203,29666.908203,29646.714844,29646.714844,0,0.0,0.0


## Extrayendo tendencias

In [47]:
def extraer_tendencias():
    # Hacemos WebScraping de la página https://coinmarketcap.com
    global df_bitcoin, precio_actual, tendencia, media_bitcoin,  algoritmo_decision
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36'}
    url = 'https://coinmarketcap.com'
    respuesta = requests.get(url, headers = headers)
    soup = BeautifulSoup(respuesta.content, features = 'lxml')
    # Obtenemos los símbolos de las criptomonedas
    posicion_BTC = soup.findAll('p', {'class': 'sc-4984dd93-0 iqdbQL coin-item-symbol'})
    # Ahora ubicamos el índice de la posición del BTC(Bitcoin)
    indice = 0
    for pos in posicion_BTC:
        if pos.getText() == 'BTC':
            break
        indice += 1
    print('El índice en la tabla para la posicion del BTC es: ', indice)
    
    # Obtenemos de la página el precio de Bitcoin
    precios = soup.findAll('div', {'class':['sc-a0353bbc-0 gDrtaY','sc-a0353bbc-0 gDrtaY fall','sc-a0353bbc-0 gDrtaY rise']})
    # Guardamos el precio en la variable precio_actual
    # con el índice que obtuvimos en el anterior bucle for
    # a su vez también quitamos el $ y la , del string 
    # y por último lo convertimos a una variable float
    precio_actual = float(precios[indice].getText().replace('$', '').replace(',', ''))
    print('El precio actual del Bitcoin en este momento es: ', precio_actual)
    
    # Obtenemos las tendencias de 1hr para el Bitcoin
    tendencias = soup.findAll('span', {'class':['sc-d55c02b-0 gUnzUB','sc-d55c02b-0 iwhBxy']})[indice]
    # Preguntamos con un if si en la classe del span esta icon-Caret-down 
    # esto quiere decir que la tendencia es igual a baja
    # de lo contrario la tendencia es igual a alta
    tendencia = ''
    if tendencias.find('span')['class'][indice] == 'icon-Caret-down':
        tendencia = 'baja'
    else:
        tendencia = 'alta'
    print('La tendencia actual del bitcoin es: ', tendencia)
    
extraer_tendencias()

El índice en la tabla para la posicion del BTC es:  0
El precio actual del Bitcoin en este momento es:  34077.13
La tendencia actual del bitcoin es:  alta
